In [21]:
import os
import urllib
import pandas as pd
import random
from statsmodels.robust import scale
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve


In [22]:
df = pd.read_csv('/Users/naina/Documents/Personal/Coding/ebay_auctions.csv')

In [24]:
df

,item,days,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
0,xbox,7,8211480551,52.99,1.201505,hanna1104,94.0,49.99,311.60
1,xbox,7,8211480551,50.99,1.203843,wrufai1,90.0,49.99,311.60
2,xbox,7,8211480551,101.99,1.204433,wrufai1,90.0,49.99,311.60
3,xbox,7,8211480551,57.00,1.708437,newberryhwt,14.0,49.99,311.60
4,xbox,7,8211480551,144.48,3.089711,miloo2005,3.0,49.99,311.60
5,xbox,7,8211480551,170.00,4.187963,fzakhour,5.0,49.99,311.60
6,xbox,7,8211480551,189.00,5.245810,bbcboi2007,1.0,49.99,311.60
7,xbox,7,8211480551,298.48,5.863021,miloo2005,3.0,49.99,311.60
8,xbox,7,8211480551,200.00,6.334815,laxwarrior07,12.0,49.99,311.60
9,xbox,7,8211480551,250.00,6.548796,aladdin987,292.0,49.99,311.60


In [40]:
def difference(x, y):
    return abs(x - y)

df['delta'] = difference(df['price'], df['openbid'])

In [55]:
byid = df.groupby('auctionid')

In [68]:
stats = {
    'bid': {
        'std_dev_bids': 'std',
        'average_bids': 'mean',
        'total_bids': 'count'
        },
    'delta': {
        'delta': 'mean'
    }
}

new_df = byid.agg(stats)

In [69]:
new_df

bid                            delta
           std_dev_bids average_bids total_bids    delta
auctionid                                               
1638843936   489.252626  1167.857143         14  1125.00
1638844284   158.771324   362.500000          4   300.00
1638844464   130.438535   554.236250         32   440.00
1638844729    29.797295   284.545455         22    95.00
1638893549    34.022052   144.500000          5    78.50
1638917885    70.624805   135.079231         26   202.50
1639226378   389.372172  1057.083333         24  1230.00
1639253454     2.886751   252.500000          4     5.00
1639309309    86.235150   191.575556         54   373.99
1639323228    65.427791   110.187500         16   184.00
1639333116   103.775935   301.734167         48   346.62
1639341131    68.365751   127.403200         50   251.52
1639364679    94.390678  1122.000000          6   195.00
1639425502     1.443376   223.250000          4   124.50
1639453840   119.919637   149.371212         33   354.00
1639672910   119.522861  5250.000000          8   400.00
1639826019   336.216050  1463.944444         18  1125.01
1639929579   417.558193   908.646471         34  1524.00
1639979107    33.028565    58.531250         32    98.50
1640179146   129.977816   303.042727         44   454.00
1640257270     8.088999    28.069231         13    31.46
1640495398   412.768657  1270.652174         23  1300.00
1640550476   105.793510   138.000000         46   299.00
1640653873    55.462465   114.366667         30   152.50
1640793161   806.322481   932.964286         28  2394.00
1640809330   206.912246   233.445000         20   559.00
1640809333   555.921817   862.739000         40  1724.00
1640936328    98.107539   199.535000         44   316.00
1641062012    38.479397  1574.663333          6   100.01
1641112143   116.156393   758.846154         13   380.00
...                 ...          ...        ...      ...
8214735736    35.822469    77.226000         10    90.51
8214745918    21.050945   151.932308         13    83.56
8214749544    30.919348    66.861111         18   109.51
8214767887    11.921806    90.892222          9    30.01
8214772603    37.356035    83.848857         35   141.50
8214772755    25.140633    70.018421         19    79.10
8214784050     8.266565    23.553333          9    21.01
8214786481    24.173591    86.312500         16    90.01
8214801298    10.170628   115.583333          6    28.50
8214801302    13.980139    21.632500         16    39.00
8214803514    28.022601    95.876154         13    77.50
8214823984    36.745589    94.600500         20   110.00
8214862356    27.970967    90.833333          9    72.50
8214864036    21.070393    85.112727         22    67.50
8214864154    12.089642    71.730000         12    34.88
8214889177    30.859769    35.824286         21    90.00
8214917158    31.281179    63.507895         19    82.50
8214999031    30.109649   118.269524         21    92.51
8215000309    42.164777    59.616000         30   126.51
8215001975    39.769424    72.432632         19   126.51
8215125069    22.142979    59.995263         19    70.07
8215129004    38.100157    45.418889         18   121.51
8215145547     6.931836   121.292857          7    28.51
8215188105    10.635298   119.244167         12    32.51
8215408023    34.979610    46.191071         28    90.02
8215558653    27.654317    92.922143         14    80.29
8215571039    31.574015   101.950000         20   100.00
8215582227    34.238413   111.131250         16    92.50
8215605488    19.136714    35.635455         11    60.01
8215610555    10.340638    22.582143         14    30.09

[634 rows x 4 columns]